In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
pd.set_option('display.max_columns', None)

data = pd.read_excel('nba_player_data.xlsx')

In [3]:
data.sample(10)

,Year,Season_type,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PTS,EFF
749,2015-16,Playoffs,2738,69,Andre Iguodala,1610612744,GSW,24,32.0,3.3,6.8,0.476,1.0,2.7,0.385,1.3,2.4,0.561,1.5,2.9,4.4,3.8,1.2,0.4,1.0,8.9,13.0
3442,2022-23,Playoffs,1629057,103,Robert Williams III,1610612738,BOS,11,20.6,2.8,3.2,0.886,0.0,0.0,0.000,0.8,1.2,0.692,1.5,4.7,6.3,1.3,0.6,1.4,0.8,6.5,14.5
3158,2022-23,Regular%20Season,1631093,65,Jaden Ivey,1610612765,DET,74,31.1,5.5,13.3,0.416,1.6,4.7,0.343,3.6,4.8,0.747,1.0,2.9,3.9,5.2,0.8,0.2,3.2,16.3,14.3
938,2016-17,Regular%20Season,203145,104,Kent Bazemore,1610612737,ATL,73,26.9,4.0,9.9,0.409,1.3,3.6,0.346,1.6,2.3,0.708,0.6,2.5,3.2,2.4,1.2,0.7,1.7,11.0,10.3
622,2015-16,Regular%20Season,203118,213,Mike Scott,1610612737,ATL,75,15.3,2.3,5.0,0.468,0.8,2.1,0.392,0.7,0.8,0.794,0.7,2.0,2.7,1.0,0.3,0.2,0.6,6.2,7.1
1163,2016-17,Playoffs,2772,58,Trevor Ariza,1610612745,HOU,11,37.5,3.7,8.8,0.423,2.1,5.5,0.377,1.2,1.3,0.929,0.7,4.4,5.1,2.1,1.3,0.2,0.8,10.7,13.4
3242,2022-23,Regular%20Season,1628978,149,Donte DiVincenzo,1610612744,GSW,72,26.3,3.3,7.5,0.435,2.1,5.3,0.397,0.8,1.0,0.817,1.1,3.4,4.5,3.5,1.3,0.1,1.6,9.4,12.9
2824,2021-22,Regular%20Season,1629001,107,De'Anthony Melton,1610612763,MEM,73,22.7,3.8,9.5,0.404,1.9,5.1,0.374,1.2,1.6,0.750,0.9,3.6,4.5,2.7,1.4,0.5,1.5,10.8,12.3
2435,2020-21,Regular%20Season,1628977,102,Hamidou Diallo,1610612765,DET,52,23.6,4.3,9.0,0.477,0.5,1.6,0.341,2.5,3.8,0.640,1.1,4.1,5.2,1.9,0.8,0.5,1.5,11.6,12.5
3043,2021-22,Playoffs,201988,97,Patty Mills,1610612751,BKN,4,18.0,2.3,4.0,0.563,1.8,3.3,0.538,0.0,0.0,0.000,0.0,1.0,1.0,0.0,0.0,0.3,0.3,6.3,5.5


In [4]:
data.shape

(3485, 27)

# Data Cleaning & Analysis preparation

In [5]:
data.drop(columns =['RANK', 'EFF'], inplace=True)

In [6]:
data['season_start_year'] = data['Year'].str[:4].astype(int)

In [7]:
data['TEAM'].replace(to_replace=['NOP', 'NOH'], value='NO', inplace=True)

In [8]:
data['Season_type'].replace('Regular%20Season', 'RS', inplace=True)

In [9]:
rs_df = data[data['Season_type']== 'RS']
playoffs_df = data[data['Season_type']== 'Playoffs']

In [10]:
data.columns

Index(['Year', 'Season_type', 'PLAYER_ID', 'PLAYER', 'TEAM_ID', 'TEAM', 'GP',
       'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PTS',
       'season_start_year'],
      dtype='object')

In [11]:
total_cols = ['MIN', 'PTS', 'AST','REB', 'STL', 'BLK', 'OREB', 'DREB',
              'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'TOV', 'GP']


# Which Player Stats are Correlated with each other?

In [12]:
data_per_min = data.groupby(['PLAYER', 'PLAYER_ID', 'Year'])[total_cols].sum().reset_index()

for col in data_per_min.columns[4:]:
    
    data_per_min['FG%'] = data_per_min['FGM']/data_per_min['FGA']
    data_per_min['3PT%'] = data_per_min['FG3M']/data_per_min['FG3A']
    data_per_min['FT%'] = data_per_min['FTM']/data_per_min['FTA']
    data_per_min['FG3A%'] = data_per_min['FG3A']/data_per_min['FGA']
    data_per_min['PTS/FGA'] = data_per_min['PTS']/data_per_min['FGA']
    data_per_min['FG3M/FGM'] = data_per_min['FG3M']/data_per_min['FGM']
    data_per_min['FTA/FGA'] = data_per_min['FTA']/data_per_min['FGA']
    data_per_min['TRU%'] = 0.5*data_per_min['PTS']/(data_per_min['FGA']+0.475*data_per_min['FTA'])
    


data_per_min = data_per_min[data_per_min['MIN']>=0]
data_per_min.drop(columns='PLAYER_ID',inplace=True)
data_per_min

,PLAYER,Year,MIN,PTS,AST,REB,STL,BLK,OREB,DREB,FGM,FGA,FG3M,FG3A,FTM,FTA,TOV,GP,FG%,3PT%,FT%,FG3A%,PTS/FGA,FG3M/FGM,FTA/FGA,TRU%
0,AJ Griffin,2022-23,19.5,8.9,1.0,2.1,0.6,0.2,0.5,1.6,3.4,7.4,1.4,3.6,0.6,0.7,0.6,72,0.459459,0.388889,0.857143,0.486486,1.202703,0.411765,0.094595,0.575493
1,Aaron Brooks,2014-15,34.0,16.1,4.1,3.5,1.0,0.3,0.7,2.9,6.0,15.1,2.2,6.0,2.1,2.7,2.7,94,0.397351,0.366667,0.777778,0.397351,1.066225,0.366667,0.178808,0.491378
2,Aaron Brooks,2015-16,16.1,7.1,2.6,1.5,0.4,0.1,0.3,1.2,2.7,6.8,1.0,2.7,0.7,0.9,1.2,69,0.397059,0.370370,0.777778,0.397059,1.044118,0.370370,0.132353,0.491180
3,Aaron Brooks,2016-17,13.8,5.0,1.9,1.1,0.4,0.1,0.3,0.8,1.9,4.6,0.7,2.0,0.5,0.6,1.0,65,0.413043,0.350000,0.833333,0.434783,1.086957,0.368421,0.130435,0.511771
4,Aaron Gordon,2015-16,23.9,9.2,1.6,6.5,0.8,0.7,2.0,4.5,3.5,7.4,0.5,1.8,1.7,2.5,0.8,78,0.472973,0.277778,0.680000,0.243243,1.243243,0.142857,0.337838,0.535662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2508,Zaza Pachulia,2016-17,32.2,11.2,2.7,9.7,1.3,0.8,3.2,6.5,4.4,8.4,0.0,0.1,2.3,2.9,2.1,85,0.523810,0.000000,0.793103,0.011905,1.333333,0.000000,0.345238,0.572744
2509,Zaza Pachulia,2017-18,14.1,5.4,1.6,4.7,0.6,0.2,1.3,3.4,2.2,3.8,0.0,0.0,1.1,1.3,1.0,69,0.578947,NaN,0.846154,0.000000,1.421053,0.000000,0.342105,0.611205
2510,Zaza Pachulia,2018-19,12.9,3.9,1.3,3.9,0.5,0.3,1.5,2.4,1.3,2.8,0.0,0.1,1.4,1.8,0.8,68,0.464286,0.000000,0.777778,0.035714,1.392857,0.000000,0.642857,0.533516
2511,Ziaire Williams,2021-22,38.5,15.0,1.5,3.7,1.1,0.2,0.8,2.9,5.4,12.0,2.3,7.5,1.9,2.2,0.9,72,0.450000,0.306667,0.863636,0.625000,1.250000,0.425926,0.183333,0.574933


In [13]:
data_per_min = data.groupby(['PLAYER', 'PLAYER_ID', 'Year'])[total_cols].sum().reset_index()

for col in data_per_min.columns[4:]:
    
    data_per_min['FG%'] = data_per_min['FGM']/data_per_min['FGA']
    data_per_min['3PT%'] = data_per_min['FG3M']/data_per_min['FG3A']
    data_per_min['FT%'] = data_per_min['FTM']/data_per_min['FTA']
    data_per_min['FG3A%'] = data_per_min['FG3A']/data_per_min['FGA']
    data_per_min['PTS/FGA'] = data_per_min['PTS']/data_per_min['FGA']
    data_per_min['FG3M/FGM'] = data_per_min['FG3M']/data_per_min['FGM']
    data_per_min['FTA/FGA'] = data_per_min['FTA']/data_per_min['FGA']
    data_per_min['TRU%'] = 0.5*data_per_min['PTS']/(data_per_min['FGA']+0.475*data_per_min['FTA'])

data_per_min[col] =  data_per_min[col]/ data_per_min['MIN']
data_per_min = data_per_min[data_per_min['MIN']>=50]
data_per_min.drop(columns='PLAYER_ID',inplace=True)

fig = px.imshow(data_per_min.corr(numeric_only=True))
fig.show()

# How are the Points Distributed?

In [14]:
fig = px.histogram(x=rs_df['PTS'], histnorm='percent')
fig.show()

In [15]:
fig = px.histogram(x=playoffs_df['PTS'], histnorm='percent')
fig.show()

In [16]:
def hist_data(df=rs_df, min_PTS=0, min_GP=0):
    return df.loc[(df['PTS']>=min_PTS) & (df['GP']>=min_GP), 'PTS'] /\
    df.loc[(df['PTS']>=min_PTS) & (df['GP']>=min_GP), 'GP']

In [17]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=hist_data(rs_df, 2,20), histnorm='percent', name='RS',
                           xbins={'start':0, 'end': 8, 'size': 1}))
fig.add_trace(go.Histogram(x=hist_data(playoffs_df, 2, 4), histnorm='percent', name='Playoffs',
                           xbins={'start':0, 'end': 8, 'size': 1}))

fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.5)
fig.show()

In [18]:
((hist_data(playoffs_df, 0, 4)>=0)&(hist_data(playoffs_df, 0, 4)<=1)).mean()

0.4637062339880444

# How has the game changed over the past 10 years?

In [19]:
change_df = data.groupby('season_start_year')[total_cols].sum().reset_index()
change_df['POSS_est'] = change_df['FGA']-change_df['OREB']+change_df['TOV']+0.44*change_df['FTA']
change_df = change_df[list(change_df.columns[0:2])+['POSS_est']+list(change_df.columns[2:-1])]

change_df['FG%'] = change_df['FGM']/change_df['FGA']
change_df['3PT%'] = change_df['FG3M']/change_df['FG3A']
change_df['FT%'] = change_df['FTM']/change_df['FTA'] 
change_df['FG3A%'] = change_df['FG3A']/change_df['FGA']
change_df['PTS/FGA'] = change_df['PTS']/change_df['FGA']
change_df['FG3M/FGM'] = change_df['FG3M']/change_df['FGM']
change_df['FTA/FGA'] = change_df['FTA']/change_df['FGA']
change_df['TRU%'] = 0.5*change_df['PTS']/(change_df['FGA']+0.475*change_df['FTA'])



change_df

,season_start_year,MIN,POSS_est,PTS,AST,REB,STL,BLK,OREB,DREB,FGM,FGA,FG3M,FG3A,FTM,FTA,TOV,GP,FG%,3PT%,FT%,FG3A%,PTS/FGA,FG3M/FGM,FTA/FGA,TRU%
0,2014,10453.0,4201.076,4375.1,946.3,1898.5,330.7,214.3,475.5,1426.9,1633.5,3648.9,345.5,987.5,766.6,1022.9,577.6,20597,0.447669,0.349873,0.749438,0.270630,1.199019,0.211509,0.280331,0.529061
1,2015,10432.3,4202.760,4382.5,916.6,1888.0,331.4,215.9,455.3,1434.1,1624.2,3638.0,372.1,1060.3,763.5,1014.0,573.9,21320,0.446454,0.350938,0.752959,0.291451,1.204645,0.229097,0.278725,0.531902
2,2016,10489.6,4313.692,4642.8,982.4,1862.9,331.6,203.1,433.4,1432.2,1701.7,3713.4,433.3,1211.5,807.5,1039.3,576.4,21191,0.458259,0.357656,0.776965,0.326251,1.250283,0.254628,0.279878,0.551786
3,2017,6293.6,2602.492,2809.8,605.1,1133.1,203.0,126.4,253.4,880.3,1046.6,2250.9,275.0,751.9,440.8,571.8,353.4,18517,0.464970,0.365740,0.770899,0.334044,1.248301,0.262756,0.254032,0.556946
4,2018,10152.9,4313.064,4659.2,1001.9,1895.8,313.1,204.3,438.7,1458.9,1701.1,3745.5,476.6,1357.1,783.9,1010.6,561.6,20015,0.454172,0.351190,0.775678,0.362328,1.243946,0.280172,0.269817,0.551315
5,2019,10354.7,4459.564,4833.1,1038.1,1873.4,316.6,195.3,398.0,1476.4,1737.8,3808.3,555.2,1535.2,805.6,1030.6,595.8,17617,0.456319,0.361647,0.781681,0.403120,1.269096,0.319484,0.270619,0.562271
6,2020,9662.4,4057.904,4525.8,964.4,1745.3,288.5,182.8,394.6,1350.5,1654.0,3545.7,508.6,1378.7,710.1,904.1,509.0,16099,0.466481,0.368898,0.785422,0.388837,1.276419,0.307497,0.254985,0.569262
7,2021,9645.7,4051.844,4444.1,977.5,1725.0,296.1,182.8,384.1,1339.6,1615.4,3507.5,506.8,1419.9,708.8,905.1,530.2,17328,0.460556,0.356927,0.783118,0.404818,1.267028,0.313730,0.258047,0.564341
8,2022,10137.9,4279.652,4816.5,1050.4,1810.9,300.1,196.3,435.9,1376.1,1761.1,3738.7,525.8,1462.7,768.0,975.8,547.5,18237,0.471046,0.359472,0.787047,0.391232,1.288282,0.298563,0.261000,0.573092


In [20]:
change_per48_df = change_df.copy()

for col in change_per48_df.columns[2:17]:
    change_per48_df[col] = change_per48_df[col]/change_df['MIN']*48*5
    
change_per48_df.drop(columns='MIN', inplace=True)
change_per48_df.drop(columns='GP', inplace=True)


fig = go.Figure()
for col in change_per48_df.columns[1:]:
    fig.add_trace(go.Scatter(x=change_per48_df['season_start_year'],
                             y=change_per48_df[col], name=col))
    
fig.show()

In [21]:
change_per100_df = change_df.copy()

for col in change_per100_df.columns[3:17]:
    change_per100_df[col] = change_per100_df[col]/change_per100_df['POSS_est']*100
    
change_per100_df.drop(columns='MIN', inplace=True)

change_per100_df

fig = go.Figure()
for col in change_per100_df.columns[1:]:
   fig.add_trace(go.Scatter(x=change_per100_df['season_start_year'],
                            y=change_per100_df[col], name=col))
    
fig.show()

# Compare Regular Season (RS) to Playoffs

In [22]:
rs_change_df = rs_df.groupby('season_start_year')[total_cols].sum().reset_index()
playoffs_change_df = playoffs_df.groupby('season_start_year')[total_cols].sum().reset_index()

for i in [rs_change_df,playoffs_change_df]:
    i['POSS_est'] = i['FGA']-i['OREB']+i['TOV']+0.44*i['FTA']
    i['POSS_per_48'] = (i['POSS_est']/i['MIN'])*48*5
    
    i['FG%'] = i['FGM']/i['FGA']
    i['3PT%'] = i['FG3M']/i['FG3A']
    i['FT%'] = i['FTM']/i['FTA'] 
    i['FG3A%'] = i['FG3A']/i['FGA']
    i['PTS/FGA'] = i['PTS']/i['FGA']
    i['FG3M/FGM'] = i['FG3M']/i['FGM']
    i['FTA/FGA'] = i['FTA']/i['FGA']
    i['TRU%'] = 0.5*i['PTS']/(i['FGA']+0.475*i['FTA'])
    
    for col in total_cols:
        i[col] = 100*i[col]/i['POSS_est']
    
    i.drop(columns=['MIN','POSS_est'], inplace=True)

rs_change_df
playoffs_change_df

,season_start_year,PTS,AST,REB,STL,BLK,OREB,DREB,FGM,FGA,FG3M,FG3A,FTM,FTA,TOV,GP,POSS_per_48,FG%,3PT%,FT%,FG3A%,PTS/FGA,FG3M/FGM,FTA/FGA,TRU%
0,2014,102.827437,22.032061,45.469143,7.727613,5.252101,11.206712,34.369480,37.969009,87.111278,8.584007,25.297061,18.506129,24.989295,13.100145,92.932077,96.283444,0.435868,0.339328,0.740562,0.290399,1.180415,0.226079,0.286866,0.519429
1,2015,102.135313,20.497032,44.643470,7.518322,5.254594,10.865897,33.805012,37.523015,86.961473,8.945157,26.211232,18.295042,24.784397,12.999289,108.384576,94.458619,0.431490,0.341272,0.738168,0.301412,1.174489,0.238391,0.285004,0.517224
2,2016,107.733544,21.849548,42.809174,7.624097,4.721893,10.134471,32.794245,39.116667,85.850791,10.254013,29.068532,19.325892,24.957629,13.302323,98.090522,96.851092,0.455635,0.352753,0.774348,0.338594,1.254893,0.262139,0.290709,0.551317
3,2018,104.768130,22.463283,44.206756,7.139744,4.662558,10.258924,34.012679,37.559855,86.779346,10.965766,32.482271,18.864229,24.240113,12.813928,96.428701,99.163303,0.432820,0.337592,0.778224,0.374309,1.207293,0.291954,0.279330,0.532936
4,2019,107.549375,22.237605,42.376964,6.827224,4.145779,8.564140,33.844519,37.888873,84.551077,13.052232,36.342129,18.858858,23.974522,13.464274,92.107298,100.659875,0.448118,0.359149,0.786621,0.429825,1.272005,0.344487,0.283551,0.560509
5,2020,111.508464,21.918484,43.319835,6.484052,4.389001,10.110613,33.222481,40.376155,88.144659,12.643239,34.654542,18.232254,23.131758,11.787981,96.133701,98.784511,0.458067,0.364836,0.788191,0.393155,1.265062,0.313136,0.262429,0.562423
6,2021,108.491402,22.609163,41.875818,7.017311,4.626082,9.473695,32.363029,38.898184,85.347960,11.930080,33.939806,18.888750,23.970925,13.578529,98.190225,98.622785,0.455760,0.351507,0.787986,0.397664,1.271166,0.306700,0.280861,0.560771
7,2022,110.792992,23.987570,43.364125,7.093375,4.720342,10.816915,32.611520,40.663112,88.702562,12.463247,35.447584,17.048538,21.627399,12.598298,69.197387,98.457493,0.458421,0.351597,0.788284,0.399623,1.249039,0.306500,0.243819,0.559699


In [23]:
comp_change_df = round(100*(playoffs_change_df-rs_change_df)/rs_change_df,3)
comp_change_df['season_start_year'] = list(range(2014,2023))
comp_change_df

,season_start_year,PTS,AST,REB,STL,BLK,OREB,DREB,FGM,FGA,FG3M,FG3A,FTM,FTA,TOV,GP,POSS_per_48,FG%,3PT%,FT%,FG3A%,PTS/FGA,FG3M/FGM,FTA/FGA,TRU%
0,2014,-1.946,-3.358,0.959,-2.815,4.672,-1.525,1.860,-3.602,0.456,6.962,12.348,2.216,4.145,-7.138,-86.906,-0.278,-4.040,-4.794,-1.853,11.838,-2.392,10.958,3.672,-2.806
1,2015,-3.110,-8.928,-0.949,-6.954,3.545,0.461,-1.419,-4.382,0.708,1.590,6.089,1.086,4.233,-7.172,-84.928,-3.528,-5.054,-4.241,-3.019,5.343,-3.791,6.245,3.500,-4.179
2,2016,0.149,-6.103,-1.333,-1.255,0.445,1.343,-1.871,-1.288,-0.416,3.236,5.483,5.065,5.621,-0.686,-86.029,-2.872,-0.876,-2.130,-0.527,5.923,0.567,4.583,6.062,-0.131
3,2017,-2.962,-3.387,1.534,-8.467,-4.001,5.362,0.554,-6.603,0.334,3.776,12.428,11.375,10.327,-5.636,-86.447,-0.081,-6.914,-7.696,0.950,12.054,-3.285,11.113,9.959,-4.311
4,2018,-2.084,-5.995,-3.281,-6.807,-13.233,-15.397,0.367,-6.418,-2.675,17.618,17.617,5.999,4.324,2.498,-86.225,-2.817,-3.846,0.001,1.607,20.850,0.608,25.685,7.191,-0.193
5,2019,2.463,-8.089,3.619,-10.544,-2.622,10.838,1.591,2.067,2.662,4.319,3.832,3.417,2.187,-11.389,-82.859,-5.835,-0.580,0.468,1.204,1.140,-0.195,2.206,-0.463,-0.143
6,2020,-2.736,-9.050,-2.220,-6.181,1.154,-0.231,-2.858,-5.096,-1.813,-4.320,1.089,10.684,10.079,4.528,-82.910,-3.329,-3.344,-5.351,0.549,2.956,-0.940,0.818,12.112,-2.200
7,2021,0.350,-4.237,0.850,-5.231,6.274,14.063,-2.615,0.503,1.597,-3.087,-0.752,2.441,1.336,-1.458,-88.991,-3.664,-1.078,-2.352,1.090,-2.312,-1.227,-3.571,-0.257,-1.201
8,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:


fig = go.Figure()
for col in comp_change_df.columns[1:]:
   fig.add_trace(go.Scatter(x=comp_change_df['season_start_year'],
                            y=comp_change_df[col], name=col))



fig.show()